In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import ttest_1samp
from scipy.special import gammaln
import scipy.stats as st
import numdifftools as nd

# Obtain data

In [2]:
symbols = ['^GSPC', 'GOOG', 'MSFT']
symbols = ['^AEX', '^BFX', '^GDAXI']
start_date = '2010-01-01'
start_date = '2000-01-01'
end_date = '2022-04-08'
vPrices = yf.download(symbols, start=start_date, end=end_date)['Close'].dropna()
vReturns = vPrices.pct_change().dropna() * 100
vReturns = np.log(vPrices / vPrices.shift(1)).dropna() *100
vNpReturns = np.array(vReturns.transpose())
vTrain = vNpReturns[:,:int(len(vNpReturns[0,:]) * 0.8)]
vTest = vNpReturns[:,int(len(vNpReturns[0,:]) * 0.8):]


[*********************100%***********************]  3 of 3 completed


In [ ]:
plt.plot(vNpReturns[0,:])

# CCC Model

### Here We will focuss on the CCC model and its loglikelihood 

In [3]:
def fnGARCH1Model(vPar, vData):
    iT = len(vData)
    dOmega = vPar[0]
    dAlpha = vPar[1]
    dBeta = vPar[2]

    vSig2 = np.zeros(iT)
    vSig2[0] = np.var(vData)

    for t in range(1,iT):
        vSig2[t] = dOmega + dAlpha * vData[t-1]**2 + dBeta * vSig2[t-1]
    return vSig2

def fnMinusNormalLogLiklihood(vPar, vData):
    vSig2 = fnGARCH1Model(vPar, vData)
    ll = -(1/2) * np.log(2*np.pi) - (1/2) * np.log(vSig2) - (1/2) * vData**2 / vSig2
    ll = np.sum(ll)
    return -ll

def fnMinusStudentTLogLiklihood(vPar, vData):
    vH = fnGARCH1Model(vPar, vData)
    dNu = vPar[3]
    ll = gammaln( (dNu+1)/2 ) - gammaln( dNu/2 ) - 0.5*np.log( (dNu-2)*np.pi*vH ) - 0.5*(dNu+1)*np.log( 1 +  (vData**2)/((dNu-2)*vH) ) 
    ll = np.sum(ll)
    return -ll

def fnMinimiseCCCNormal(vData, vIni):
    results = minimize(fnMinusNormalLogLiklihood, vIni, args= vData, method= "Nelder-Mead")
    results = [np.exp(results.x[0]), np.exp(results.x[1])/(1+np.exp(results.x[1])), np.exp(results.x[2])/(1+np.exp(results.x[2]))]
    return results

def fnMinimiseCCCStudentT(vData, vIni):
    results = minimize(fnMinusStudentTLogLiklihood, vIni, args= vData, method= "Nelder-Mead")
    results = [np.exp(results.x[0]), np.exp(results.x[1])/(1+np.exp(results.x[1])), np.exp(results.x[2])/(1+np.exp(results.x[2]))]
    return results

def fnThreeDimMulti(m1, m2):
    m3 = np.zeros(m1.shape)
    for t in range(m1.shape[2]):
        m3[:,:,t] = m1[:,:,t] @ m2[:,:,t]
    return m3

In [18]:
# One step code

def fnCCCOneStep(vPar, mData):
    iN = mData.shape[0]
    iT = mData.shape[1]
    mDt = np.zeros((mData.shape[0], mData.shape[0], mData.shape[1]))
    mSigt = np.zeros((mData.shape[0], mData.shape[0], mData.shape[1]))
    mR = np.identity(mData.shape[0])
    
    p = 0
    #print(vPar[0:3])
    for i in range(iN):
        mDt[i, i, :] = np.sqrt(fnGARCH1Model(vPar[i*3 : (i+1)*3], mData[i, :]))
        for j in range(iN):
            if i < mData.shape[0] & j > i:
                mR[i,j] = vPar[iN*3 + p]
                mR[j,i] = vPar[iN*3 + p]
                p += 1
    dLL = 0
    for t in range(iT):
        mSigt[:,:,t] = mDt[:,:,t] @ mR @ mDt[:,:,t]
        dLL += iN*np.log(2*np.pi) + np.log(np.linalg.det(mSigt[:,:,t])) + mData[:,t].transpose() @ np.linalg.inv(mSigt[:,:,t]) @ mData[:,t]
    dLL = -(1/2)*dLL
    print(dLL)
    return -dLL 


In [13]:
bounds = [(0.00000001, 20), (0.00000001, 0.99999999), (0.00000001, 0.99999999)]*3 + [(-1, 1)]*3


options = {'maxiter': 10000, 'adaptive':True}
results = minimize(fnCCCOneStep, [np.var(vTrain[0,:])*(1-0.30-0.30), 0.30, 0.30, np.var(vTrain[1,:])*(1-0.30-0.30), 0.30, 0.30, np.var(vTrain[2,:])*(1-0.30-0.30), 0.30, 0.30, 0.85, 0.85, 0.77], bounds= bounds, options= options, args= vTrain, method= "SLSQP")
results

C:\Users\onim-\AppData\Local\Temp\ipykernel_24176\3167218802.py:5: OptimizeWarning: Unknown solver options: adaptive
  results = minimize(fnCCCOneStep, [np.var(vTrain[0,:])*(1-0.30-0.30), 0.30, 0.30, np.var(vTrain[1,:])*(1-0.30-0.30), 0.30, 0.30, np.var(vTrain[2,:])*(1-0.30-0.30), 0.30, 0.30, 0.85, 0.85, 0.77], bounds= bounds, options= options, args= vTrain, method= "SLSQP")


-16245.621906516302
-16245.621913910842
-16245.621908433282
-16245.621905179532
-16245.621909874992
-16245.621906588176
-16245.62190382098
-16245.621907242607
-16245.621906158583
-16245.621897878136
-16245.621910398071
-16245.621871036112
-16245.6218959971
nan


C:\Users\onim-\AppData\Local\Temp\ipykernel_24176\2688718809.py:22: RuntimeWarning: invalid value encountered in log
  dLL += iN*np.log(2*np.pi) + np.log(np.linalg.det(mSigt[:,:,t])) + mData[:,t].transpose() @ np.linalg.inv(mSigt[:,:,t]) @ mData[:,t]


-16981.839986093455
-16243.204098549128
-16238.905402902255
-16238.905410973835
-16238.905405051742
-16238.9054030855
-16238.905407611572
-16238.905403675219
-16238.905402023747
-16238.905403897126
-16238.905403268043
-16238.905394986168
-16238.905389337679
-16238.905365231481
-16238.905397244136
nan
-16110.215646635039
-16110.215651997441
-16110.215639648928
-16110.215638793756
-16110.215650022436
-16110.215643500971
-16110.21564277521
-16110.215652069537
-16110.215656798351
-16110.21565298235
-16110.215639667278
-16110.21560976576
-16110.215644955008
nan
-16003.06109386468
-16003.061095045854
-16003.061073513174
-16003.06107402657
-16003.061104395323
-16003.061103569531
-16003.061105918214
-16003.061098503365
-16003.061102208343
-16003.061097821712
-16003.061097808557
-16003.061068317413
-16003.06109091499
nan
-16111.679399182653
-15970.41044653068
-15970.41044998139
-15970.410433259069
-15970.410432059452
-15970.410457345384
-15970.410456620219
-15970.410459607765
-15970.41045070486

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 14824.957593812696
       x: [ 1.614e-02  6.323e-02  9.242e-01  2.239e-02  7.346e-02
            9.062e-01  1.844e-02  5.890e-02  9.299e-01  8.339e-01
            8.790e-01  7.912e-01]
     nit: 51
     jac: [ 4.270e-01  2.740e-01  3.959e-01  1.964e-01 -1.821e-01
           -1.641e-01 -5.389e-01 -7.703e-02 -2.006e-01  3.613e-02
           -3.333e-02  2.917e-02]
    nfev: 717
    njev: 51

In [ ]:
test = results.x

In [19]:
def NormalCopula(vPar, mData):
    iN = mData.shape[0]
    iT = mData.shape[1]
    vLL = np.zeros(iN)
    mI = np.identity(iN)
    mR = np.identity(mData.shape[0])
    mSig = np.zeros(mData.shape)
    
    p = 0
    #print(vPar[0:3])
    for i in range(iN):
        vLL[i] = -1 * fnMinusNormalLogLiklihood(vPar[i*3 : (i+1)*3], mData[i, :])
        mSig[i,:] = fnGARCH1Model(vPar[i*3 : (i+1)*3], mData[i, :])
        for j in range(iN):
            if i < mData.shape[0] & j > i:
                mR[i,j] = vPar[iN*3 + p]
                mR[j,i] = vPar[iN*3 + p]
                p += 1
                
    mUt = st.norm.cdf(mData, loc=np.zeros(mData.shape), scale=np.sqrt(mSig))
    dLL = 0 
    for t in range(iT):
        vPpf_t = st.norm.ppf(mUt[:,t]).reshape(iN, 1)
        dLL += -0.5 * np.log(np.linalg.det(mR)) - 0.5 * np.transpose(vPpf_t) @ (np.linalg.inv(mR) - mI) @ vPpf_t

    print((dLL + np.sum(vLL))[0][0])
    if vPar[1] + vPar[2] > 1:
        dLL = - np.inf
    elif vPar[4] + vPar[5] > 1:
        dLL = - np.inf
    elif vPar[7] + vPar[8] > 1:
        dLL = -np.inf
    return -(dLL + np.sum(vLL))

In [20]:
def NormalCopula_LL(vPar, mData):
    iN = mData.shape[0]
    iT = mData.shape[1]
    vLL = np.zeros(iN)
    mI = np.identity(iN)
    mR = np.identity(mData.shape[0])
    mSig = np.zeros(mData.shape)
    
    p = 0
    #print(vPar[0:3])
    for i in range(iN):
        vLL[i] = -1 * fnMinusNormalLogLiklihood(vPar[i*3 : (i+1)*3], mData[i, :])
        mSig[i,:] = fnGARCH1Model(vPar[i*3 : (i+1)*3], mData[i, :])
        for j in range(iN):
            if i < mData.shape[0] & j > i:
                mR[i,j] = vPar[iN*3 + p]
                mR[j,i] = vPar[iN*3 + p]
                p += 1
                
    mUt = st.norm.cdf(mData, loc=np.zeros(mData.shape), scale=np.sqrt(mSig))
    dLL = 0 
    for t in range(iT):
        vPpf_t = st.norm.ppf(mUt[:,t]).reshape(iN, 1)
        dLL += -0.5 * np.log(np.linalg.det(mR)) - 0.5 * np.transpose(vPpf_t) @ (np.linalg.inv(mR) - mI) @ vPpf_t
    return -(dLL + np.sum(vLL))[0][0]

In [20]:
results.x

array([0.0161401 , 0.06322516, 0.92416926, 0.02238642, 0.0734586 ,
       0.90619564, 0.01843834, 0.05890423, 0.92985514, 0.83386119,
       0.87895546, 0.79115102])

In [42]:
bounds = [(0.00001, 50), (0.00000001, 0.99999999), (0.00000001, 0.99999999)]*3 + [(-1, 1)]*3
ini = [1, 0.30, 0.30, 1, 0.30, 0.30, 1, 0.30, 0.30, 0.85, 0.85, 0.75]
ini1 = results.x
# ini2 = [0.016, 0.1, 0.8, 0.2, 0.11,  0.7, 0.1, 0.2, 0.7, 0.85, 0.85, 0.75]
#options = {'ftol': 1e-1000}
options = {'maxiter': 10000, 'adaptive': True}
results_Copula = minimize(NormalCopula, ini, args= vTrain, method= "SLSQP", options= options, bounds = bounds)
results_Copula

C:\Users\onim-\AppData\Local\Temp\ipykernel_24176\2315311956.py:7: OptimizeWarning: Unknown solver options: adaptive
  results_Copula = minimize(NormalCopula, ini, args= vTrain, method= "SLSQP", options= options, bounds = bounds)


-16597.692859311366
-16597.69286574218
-16597.692860199062
-16597.69285915399
-16597.692873254975
-16597.692868692164
-16597.69288230141
-16597.69285941282
-16597.692857825772
-16597.69284998807
-16597.692830443113
-16597.692816847397
-16597.692840775897


C:\Users\onim-\AppData\Local\Temp\ipykernel_24176\3206308289.py:24: RuntimeWarning: invalid value encountered in matmul
  dLL += -0.5 * np.log(np.linalg.det(mR)) - 0.5 * np.transpose(vPpf_t) @ (np.linalg.inv(mR) - mI) @ vPpf_t


nan
-16338.69944742124
-16338.699455133024
-16338.699446489125
-16338.699449623586
-16338.699454064099
-16338.699446797704
-16338.699450856751
-16338.699451616776
-16338.699455307062
-16338.699451527458
-16338.699444293929
-16338.699407956858
-16338.699424289887


C:\Users\onim-\AppData\Local\Temp\ipykernel_24176\3206308289.py:24: RuntimeWarning: invalid value encountered in log
  dLL += -0.5 * np.log(np.linalg.det(mR)) - 0.5 * np.transpose(vPpf_t) @ (np.linalg.inv(mR) - mI) @ vPpf_t


nan
-16271.857712170508
-16271.857720790464
-16271.85772127366
-16271.857718057125
-16271.857705538325
-16271.857695732422
-16271.857695557357
-16271.857706955456
-16271.857706796996
-16271.857690875357
-16271.857767824695
-16271.857719158452
-16271.85768886819
nan
-16204.463183885451
-16204.463178377984
-16204.463170597324
-16204.463153018361
-16204.463186401379
-16204.463183862357
-16204.463178824275
-16204.463183285858
-16204.463185500697
-16204.463175937866
-16204.463214651803
-16204.4632230475
-16204.463189453098
nan
-18192.62577076361
-16306.923774850602
-16176.906432071168
-16176.90642930776
-16176.906422659991
-16176.906408292069
-16176.906434512919
-16176.906431751395
-16176.906426627425
-16176.906436413894
-16176.906437943982
-16176.906434503691
-16176.906428366303
-16176.90642066047
-16176.906453955842
nan
-16917.356074342664
-16170.97213740203
-16170.97213290347
-16170.97212489885
-16170.972109614091
-16170.972142949264
-16170.972140109743
-16170.972137576397
-16170.9721447

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 14824.957593839703
       x: [ 1.614e-02  6.323e-02  9.242e-01  2.239e-02  7.346e-02
            9.062e-01  1.844e-02  5.890e-02  9.299e-01  8.339e-01
            8.790e-01  7.912e-01]
     nit: 70
     jac: [-5.891e-01 -2.485e-01 -4.879e-01  6.364e-01  6.237e-01
            7.977e-01  2.694e-01  7.678e-02  2.682e-01 -1.843e-02
            3.015e-02 -3.040e-02]
    nfev: 983
    njev: 68

In [43]:
results.x

array([0.0161401 , 0.06322516, 0.92416926, 0.02238642, 0.0734586 ,
       0.90619564, 0.01843834, 0.05890423, 0.92985514, 0.83386119,
       0.87895546, 0.79115102])

In [44]:
results_Copula.x

array([0.01614028, 0.06322531, 0.92416935, 0.02238617, 0.07345948,
       0.90619501, 0.01843836, 0.05890489, 0.92985459, 0.83386142,
       0.87895595, 0.79115158])

In [33]:
mHessian = nd.Hessian(NormalCopula_LL, step= 1e-5)(results_Copula.x, mData = vTrain)
mCov1 = np.linalg.inv( mHessian )
np.sqrt(mCov1.diagonal()) 

array([0.00204995, 0.00412075, 0.00469038, 0.0029048 , 0.00558871,
       0.00700516, 0.00247579, 0.00403847, 0.00456634, 0.00455456,
       0.00336808, 0.00554328])

In [48]:
vCopulaNormalParam = [0.0161401 , 0.06322516, 0.92416926, 0.02238642, 0.0734586 ,
       0.90619564, 0.01843834, 0.05890423, 0.92985514, 0.83386119,
       0.87895546, 0.79115102]

In [8]:
vCopulaNormalParamSE = [0.00204995, 0.00412075, 0.00469038, 0.0029048 , 0.00558871,
       0.00700516, 0.00247579, 0.00403847, 0.00456634, 0.00455456,
       0.00336808, 0.00554328]

In [38]:
np.array(vCopulaNormalParam).round(4)

array([0.0161, 0.0632, 0.9242, 0.0224, 0.0735, 0.9062, 0.0184, 0.0589,
       0.9299, 0.8339, 0.879 , 0.7912])

In [36]:
np.array(vCopulaNormalParamSE).round(4)

array([0.002 , 0.0041, 0.0047, 0.0029, 0.0056, 0.007 , 0.0025, 0.004 ,
       0.0046, 0.0046, 0.0034, 0.0055])

# AIC BIC STUFF

In [17]:
def fnAIC(k, dLL):
    return 2*k -2*dLL

def fnBIC(k, dLL, vData = vTrain):
    return k*np.log(len(vData[0,:])) -2*dLL 

In [18]:
vIC_CCC_Copula=['CCC-Copula-N',len(vCopulaNormalParam), NormalCopula_LL(vCopulaNormalParam, vTrain)*-1, fnAIC(len(vCopulaNormalParam), NormalCopula_LL(vCopulaNormalParam, vTrain)*-1), fnBIC(len(vCopulaNormalParam), NormalCopula_LL(vCopulaNormalParam, vTrain)*-1)]

In [19]:
df_IC = pd.DataFrame(columns=['Model','Number Of Parameters', 'Log Likelihood', 'AIC', 'BIC'], data=[vIC_CCC_Copula])

In [20]:
print(df_IC.to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
       Model &  Number Of Parameters &  Log Likelihood &          AIC &          BIC \\
\midrule
CCC-Copula-N &                    12 &   -14824.957594 & 29673.915188 & 29750.886477 \\
\bottomrule
\end{tabular}



C:\Users\onim-\AppData\Local\Temp\ipykernel_43604\2965693198.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_IC.to_latex(index=False))


# Forecast

In [32]:
def NormalCopula_Prediction(vPar, mData):
    iN = mData.shape[0]
    iT = mData.shape[1]
    vLL = np.zeros(iN)
    mI = np.identity(iN)
    mR = np.identity(mData.shape[0])
    mSig = np.zeros(mData.shape)
    mDt = np.zeros((mData.shape[0], mData.shape[0], mData.shape[1]))
    mSigt = np.zeros((mData.shape[0], mData.shape[0], mData.shape[1]))
    p = 0
    #print(vPar[0:3])
    for i in range(iN):
        vLL[i] = -1 * fnMinusNormalLogLiklihood(vPar[i*3 : (i+1)*3], mData[i, :])
        mSig[i,:] = fnGARCH1Model(vPar[i*3 : (i+1)*3], mData[i, :])
        mDt[i, i, :] = np.sqrt(mSig[i,:])
        for j in range(iN):
            if i < mData.shape[0] & j > i:
                mR[i,j] = vPar[iN*3 + p]
                mR[j,i] = vPar[iN*3 + p]
                p += 1
               
    for t in range(iT):
        mSigt[:,:,t] = mDt[:,:,t] @ mR @ mDt[:,:,t]
    return mSigt

In [33]:
mPred_Copula_N = NormalCopula_Prediction(vCopulaNormalParam, vTest)

In [27]:
np.save('mPred_Copula_N.npy', mPred_Copula_N)

# LSR

In [29]:
def fnMinusNormalLogLiklihoodLSR(vPar, vData):
    vSig2 = fnGARCH1Model(vPar, vData)
    ll = -(1/2) * np.log(2*np.pi) - (1/2) * np.log(vSig2) - (1/2) * vData**2 / vSig2
    return ll

In [62]:
def fn_NormalCopula_LSR(vPar, mData):
    iN = mData.shape[0]
    iT = mData.shape[1]
    vLL = np.zeros((iN, iT))
    mI = np.identity(iN)
    mR = np.identity(mData.shape[0])
    mSig = np.zeros(mData.shape)
    
    p = 0
    #print(vPar[0:3])
    for i in range(iN):
        vLL[i, :] = fnMinusNormalLogLiklihoodLSR(vPar[i*3 : (i+1)*3], mData[i, :])
        mSig[i,:] = fnGARCH1Model(vPar[i*3 : (i+1)*3], mData[i, :])
        for j in range(iN):
            if i < mData.shape[0] & j > i:
                mR[i,j] = vPar[iN*3 + p]
                mR[j,i] = vPar[iN*3 + p]
                p += 1
                
    mUt = st.norm.cdf(mData, loc=np.zeros(mData.shape), scale=np.sqrt(mSig))
    vLL1 = np.zeros(iT) 
    for t in range(iT):
        vPpf_t = st.norm.ppf(mUt[:,t]).reshape(iN, 1)
        vLL1[t] = -0.5 * np.log(np.linalg.det(mR)) - 0.5 * np.transpose(vPpf_t) @ (np.linalg.inv(mR) - mI) @ vPpf_t
    
    vTotLL = vLL1 + vLL.sum(axis=0)
    return vTotLL

In [66]:
vLSR_Copula_N = fn_NormalCopula_LSR(vCopulaNormalParam, vTest)

In [67]:
np.sum(vLSR_Copula_N)

-3345.4112829125643

In [68]:
np.save('vLSR_Copula_n.npy', vLSR_Copula_N)